# **Face Recognition Project**

### Загрузка данных

In [2]:
import requests
from urllib.parse import urlencode

In [3]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/S8f03spLIA1wrw'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('downloaded_file.zip', 'wb') as f:
    f.write(download_response.content)

Разархивируем downloaded_file.zip

In [5]:
import zipfile

with zipfile.ZipFile('downloaded_file.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files')

Удалим ненужные файлы

In [6]:
import os

filename = 'downloaded_file.zip'

# Если файл существует, удаляем
if os.path.exists(filename):
    os.remove(filename)
    print(filename + " deleted")

downloaded_file.zipdeleted


In [8]:
import shutil

filename = '/content/extracted_files/__MACOSX'

# Если файл существует, удаляем
if os.path.exists(filename):
    shutil.rmtree(filename)
    print(filename + " deleted")

/content/extracted_files/__MACOSX deleted


    Внутри:
            - celebA_imgs — папка с выровненными картинками;
            - celebA_anno.txt — файл с аннотацией — каждой картинке из celebA_imgs поставлен в соответствие ее id;
            - celebA_train_split.txt — файл со сплитом на train/val/test.